In [3]:
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import hstack
import warnings
import os
from fredapi import Fred
warnings.filterwarnings('ignore')

import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
import pmdarima as pm
from pmdarima import model_selection

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

%matplotlib inline

Using TensorFlow backend.


In [4]:
# NEED TO MINMAX SCALE

In [5]:
start = datetime(2000,1,31)
end = datetime.now()

In [6]:
AMT = web.DataReader('AMT', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')

In [7]:
fred = Fred(api_key = 'aec5f13aa40d1af4edd587ffccceeb1a')
libor = round(fred.get_series('USD1MTD156N', start, end), 2)

In [8]:
libor = libor[libor.index.isin(AMT.index)]

In [9]:
libor

2000-02-03    5.90
2000-02-04    5.89
2000-02-07    5.89
2000-02-08    5.89
2000-02-09    5.89
              ... 
2020-01-23    1.66
2020-01-24    1.66
2020-01-27    1.65
2020-01-28    1.65
2020-01-29    1.65
Length: 5028, dtype: float64

In [10]:
AMT.drop(AMT.tail(5).index, inplace= True)

In [11]:
AMT['mid'] = (AMT['low']+AMT['high'])/2

## LSTM Model

In [12]:
# Define input sequences
amt_seq = np.array(AMT['mid'])
libor_seq = np.array(libor)
out_seq = np.array([amt_seq[i] + libor_seq[i] for i in range(len(amt_seq))])

In [13]:
# convert to [rows, columns] structure
amt_seq = amt_seq.reshape((len(amt_seq), 1))
libor_seq = libor_seq.reshape((len(libor_seq), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((amt_seq, libor_seq, out_seq))

In [14]:
print(dataset)

[[ 39.44     5.9     45.34  ]
 [ 40.905    5.89    46.795 ]
 [ 38.655    5.89    44.545 ]
 ...
 [236.9175   1.65   238.5675]
 [237.       1.65   238.65  ]
 [235.36     1.65   237.01  ]]


In [15]:
# Split multivariate sequence into samples
# The split_sequences() function will split multiple parallel time series with rows for time steps 
# and one series per column into the required input/output shape.

def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences) - 1:
            break
        seq_x, seq_y = sequences[i:end_ix,:], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [16]:
n_steps = 3

X, y = split_sequences(dataset, n_steps)
n_features = X.shape[2]
print(X.shape, y.shape)

(5025, 3, 3) (5025, 3)


In [17]:
for i in range(len(X)):
    print(X[i], y[i])

[[39.44   5.9   45.34 ]
 [40.905  5.89  46.795]
 [38.655  5.89  44.545]] [40.44  5.89 46.33]
[[40.905  5.89  46.795]
 [38.655  5.89  44.545]
 [40.44   5.89  46.33 ]] [41.86  5.89 47.75]
[[38.655  5.89  44.545]
 [40.44   5.89  46.33 ]
 [41.86   5.89  47.75 ]] [43.25  5.89 49.14]
[[40.44  5.89 46.33]
 [41.86  5.89 47.75]
 [43.25  5.89 49.14]] [44.53  5.88 50.41]
[[41.86  5.89 47.75]
 [43.25  5.89 49.14]
 [44.53  5.88 50.41]] [44.345  5.88  50.225]
[[43.25   5.89  49.14 ]
 [44.53   5.88  50.41 ]
 [44.345  5.88  50.225]] [46.125  5.88  52.005]
[[44.53   5.88  50.41 ]
 [44.345  5.88  50.225]
 [46.125  5.88  52.005]] [45.875  5.88  51.755]
[[44.345  5.88  50.225]
 [46.125  5.88  52.005]
 [45.875  5.88  51.755]] [44.94  5.88 50.82]
[[46.125  5.88  52.005]
 [45.875  5.88  51.755]
 [44.94   5.88  50.82 ]] [43.565  5.88  49.445]
[[45.875  5.88  51.755]
 [44.94   5.88  50.82 ]
 [43.565  5.88  49.445]] [41.72  5.88 47.6 ]
[[44.94   5.88  50.82 ]
 [43.565  5.88  49.445]
 [41.72   5.88  47.6  ]] [42

In [24]:
#Define Model
model = Sequential()
model.add(LSTM(50, activation = 'relu', return_sequences = True, input_shape = (n_steps, n_features)))
model.add(LSTM(50, activation = 'relu'))
model.add(Dense(n_features))
model.compile(optimizer = 'adam', loss = ['mse'])

In [25]:
model.fit(X, y, epochs = 50, batch_size = 30, verbose = 1)

Epoch 1/50
5025/5025 [==============================] - 1s 289us/step - loss: nan
Epoch 2/50
5025/5025 [==============================] - 1s 156us/step - loss: nan
Epoch 3/50
5025/5025 [==============================] - 1s 161us/step - loss: nan
Epoch 4/50
5025/5025 [==============================] - 1s 154us/step - loss: nan
Epoch 5/50
5025/5025 [==============================] - 1s 153us/step - loss: nan
Epoch 6/50
5025/5025 [==============================] - 1s 151us/step - loss: nan
Epoch 7/50
5025/5025 [==============================] - 1s 152us/step - loss: nan
Epoch 8/50
5025/5025 [==============================] - 1s 155us/step - loss: nan
Epoch 9/50
5025/5025 [==============================] - 1s 153us/step - loss: nan
Epoch 10/50
5025/5025 [==============================] - 1s 161us/step - loss: nan
Epoch 11/50
5025/5025 [==============================] - 1s 163us/step - loss: nan
Epoch 12/50
5025/5025 [==============================] - 1s 154us/step - loss: nan
Epoch 13/50
5

KeyboardInterrupt: 